In [1]:
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
from sklearn import metrics
from sklearn.metrics import confusion_matrix
%matplotlib inline
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import numpy as np # linear algebra
import pandas as pd #data processing

import os
import re
import nltk

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
print(train.shape, test.shape)

(20800, 5) (5200, 4)


In [4]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
print(train.isnull().sum())
print('************')
print(test.isnull().sum())

id           0
title      558
author    1957
text        39
label        0
dtype: int64
************
id          0
title     122
author    503
text        7
dtype: int64


In [6]:
test=test.fillna(' ')
train=train.fillna(' ')
test['total']=test['title']+' '+test['author']+test['text']
train['total']=train['title']+' '+train['author']+train['text']

# Cleaning and Preprocessing

In [7]:
nltk.download('punkt')
nltk.download('wordnet')

stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sujoy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sujoy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
lemmatizer=WordNetLemmatizer()
for index,row in train.iterrows():
    filter_sentence = ''
    
    sentence = row['total']
    sentence = re.sub(r'[^\w\s]','',sentence) #cleaning
    
    words = nltk.word_tokenize(sentence) #tokenization
    
    words = [w for w in words if not w in stop_words]  #stopwords removal
    
    for word in words:
        filter_sentence = filter_sentence + ' ' + str(lemmatizer.lemmatize(word)).lower()
        
    train.loc[index,'total'] = filter_sentence

In [9]:
train = train[['total','label']]

# Applying NLP Techniques

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
X_train = train['total']
Y_train = train['label']

In [12]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(X_train)
freq_term_matrix = count_vectorizer.transform(X_train)
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(freq_term_matrix)
tf_idf_matrix = tfidf.transform(freq_term_matrix)

In [13]:
print(tf_idf_matrix)

  (0, 193339)	0.014251723682766018
  (0, 192918)	0.04798814005493949
  (0, 192788)	0.009513835689569556
  (0, 191427)	0.0432193826821374
  (0, 191415)	0.06964967740813574
  (0, 191271)	0.036826985685343996
  (0, 191184)	0.012957237699274633
  (0, 190482)	0.016032396829821688
  (0, 190280)	0.05870983669845154
  (0, 189112)	0.014844805453674404
  (0, 189028)	0.02060588499044508
  (0, 188795)	0.01669874420805987
  (0, 188537)	0.012190788273372785
  (0, 188393)	0.032903159599635246
  (0, 188266)	0.05870983669845154
  (0, 188009)	0.02334413943504067
  (0, 187910)	0.011531559308095725
  (0, 186479)	0.02785428676690503
  (0, 186446)	0.02107578335112647
  (0, 186373)	0.016565700636845072
  (0, 184878)	0.02107578335112647
  (0, 182981)	0.0643241498956477
  (0, 182075)	0.05649521247745022
  (0, 181755)	0.03386580341770357
  (0, 181390)	0.01712755096818207
  :	:
  (20799, 11097)	0.020369382803188407
  (20799, 11093)	0.02579193802780123
  (20799, 11007)	0.030318786434671566
  (20799, 10917)	0.0380

# Modelling 

In [14]:
test_counts = count_vectorizer.transform(test['total'].values)
test_tfidf = tfidf.transform(test_counts)

#split in samples
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tf_idf_matrix, Y_train, random_state=0)

In [15]:
print(X_train.shape,X_test.shape)

(15600, 220387) (5200, 220387)


In [16]:
#Logistic Regression

In [17]:
from sklearn.linear_model import LogisticRegression# Logistic Regression

In [18]:
logreg = LogisticRegression(C=1e5)
logreg.fit(X_train, y_train)
pred = logreg.predict(X_test)
print('Accuracy of Logistic classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic classifier on test set: {:.2f}'
     .format(logreg.score(X_test, y_test)))
from sklearn.naive_bayes import MultinomialNB
cm = confusion_matrix(y_test, pred)
print(cm)

Accuracy of Logistic classifier on training set: 1.00
Accuracy of Logistic classifier on test set: 0.98
[[2493   71]
 [  44 2592]]


In [19]:
#Multinomial Naive Bayes

In [20]:
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()
NB.fit(X_train, y_train)
pred = NB.predict(X_test)
print('Accuracy of NB  classifier on training set: {:.2f}'
     .format(NB.score(X_train, y_train)))
print('Accuracy of NB classifier on test set: {:.2f}'
     .format(NB.score(X_test, y_test)))
cm = confusion_matrix(y_test, pred)
print(cm)

Accuracy of NB  classifier on training set: 0.88
Accuracy of NB classifier on test set: 0.83
[[2558    6]
 [ 853 1783]]


In [21]:
#Support Vector Machine

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

classifier = SVC(kernel='rbf', random_state = 0)
classifier.fit(X_train, y_train)

Y_Pred = classifier.predict(X_test)
print("The accuracy attained using Support Vector Machine is " + str(accuracy_score(y_test, Y_Pred) * 100) + " percent")

print('Accuracy of Logistic classifier on training set: {:.2f}'.format(classifier.score(X_train, y_train)))
print('Accuracy of Logistic classifier on test set: {:.2f}'.format(classifier.score(X_test, y_test)))

cm = confusion_matrix(y_test, pred)
print(cm)

# Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
import joblib
from sklearn import linear_model

In [ ]:
pipeline = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer(norm='l2')),('clf', linear_model.LogisticRegression(C=1e5)),])

In [ ]:
pipeline.fit(X_train, Y_train)

In [ ]:
Prediction = pipeline.predict(["flynn hillary clinton big woman campus breitbart daniel j flynnever get feeling life circle roundabout rather head straight line toward intended destination hillary clinton remains big woman campus leafy liberal wellesley massachusetts everywhere else vote likely inauguration dress remainder day way miss havisham forever wore wedding dress speaking great expectations hillary rodham overflowed 48 year ago first addressed wellesley graduating class the president college informed gathered 1969 student needed debate far i could ascertain spokesman kind like democratic primary 2016 minus term unknown even seven sisters school i glad miss adams made clear i speaking today u 400 u miss rodham told classmate after appointing edger bergen charlie mccarthys mortimer snerds attendance bespectacled granny glass awarding matronly wisdom least john lennon wisdom took issue previous speaker despite becoming first win election seat u s senate since reconstruction edward brooke came criticism calling empathy goal protestors criticized tactic though clinton senior thesis saul alinsky lamented black power demagogue elitist arrogance repressive intolerance within new left similar word coming republican necessitated brief rebuttal trust rodham ironically observed 1969 one word i asked class rehearsal wanted say everyone came said talk trust talk lack trust u way feel others talk trust bust what say what say feeling permeates generation perhaps even understood distrusted the trust bust certainly busted clintons 2016 plan she certainly even understand people distrusted after whitewater travelgate vast conspiracy benghazi missing email clinton found distrusted voice friday there load compromising road broadening political horizon and distrust american people trump edged 48 percent 38 percent question immediately prior novembers election stood major reason closing horizon clinton described vanquisher supporter embracing lie con alternative fact assault truth reason she failed explain american people chose lie truth as history major among today know well people power invent fact attack question mark beginning end free society offered that hyperbole like many people emerge 1960s hillary clinton embarked upon long strange trip from high school goldwater girl wellesley college republican president democratic politician clinton drank time place gave degree more significantly went idealist cynic comparison two wellesley commencement address show way back lamented long leader viewed politics art possible challenge practice politics art making appears impossible possible now big woman campus odd woman white house wonder current station even possible why arent i 50 point ahead asked september in may asks isnt president the woman famously dubbed congenital liar bill safire concludes lie mind getting stood election day like finding jilted bride wedding day inspires dangerous delusion"])

In [ ]:
print(Prediction)